<a href="https://colab.research.google.com/github/iakovkh/mypython/blob/main/LSTM_example31.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Configuration Section
data_file_name = '/content/sample_data/NN_cut2.xlsx'
validate_file_name = '/content/sample_data/NN_control_02.xlsx'
normalize_data = True
train_split = 0.8
optimizer_name = 'Adam'  # Other options: 'SGD', 'RMSprop', 'Adagrad', etc.
loss_function_name = 'MSELoss'  # Other options: 'CrossEntropyLoss', 'NLLLoss', etc.
activation_function_name = 'ReLU'  # Other options: 'Sigmoid', 'Tanh', etc.
epochs = 1000
network_structure = [(1, 32), (2, 3), (3, 5), (4, 2)]

# Step I - General data preparation
class MyMatrix:
    def __init__(self):
        self.data = None

    def read_data_from_file(self, file_name):
        self.data = pd.read_excel(file_name)

    def normalize_data(self):
        scaler = MinMaxScaler()
        self.data.iloc[:, :] = scaler.fit_transform(self.data)

    def display_console(self, column=None):
        if column:
            print(self.data[column])
        else:
            print(self.data)

    def display_visual(self, column=None):
        if column:
            self.data[column].plot()
            plt.show()
        else:
            self.data.plot(subplots=True)
            plt.show()

# Step II – Read and prepare data
MyData = MyMatrix()
MyData.read_data_from_file(data_file_name)

if normalize_data:
    MyData.normalize_data()

train_size = int(len(MyData.data) * train_split)
train_data = MyData.data.iloc[:train_size, :]
test_data = MyData.data.iloc[train_size:, :]

# Step III – Design the neural network
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size):
        super(NeuralNetwork, self).__init__()
        layers = []
        in_size = input_size
        for layer, out_size in hidden_layers:
            layers.append(nn.Linear(in_size, out_size))
            layers.append(getattr(nn, activation_function_name)())
            in_size = out_size
        layers.append(nn.Linear(in_size, output_size))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

def display_network_structure():
    print("Network Structure:")
    for layer, neurons in network_structure:
        print(f"Layer {layer}: {neurons} neurons")
    print("Training Data:")
    print(train_data)
    print("Testing Data:")
    print(test_data)

input_size = train_data.shape[1] - 2
output_size = 2
hidden_layers = [(layer, neurons) for layer, neurons in network_structure if layer != len(network_structure)]

model = NeuralNetwork(input_size, hidden_layers, output_size)
criterion = getattr(nn, loss_function_name)()
optimizer = getattr(optim, optimizer_name)(model.parameters())

# Training the network
train_inputs = torch.tensor(train_data.iloc[:, 2:].values, dtype=torch.float32)
train_outputs = torch.tensor(train_data.iloc[:, :2].values, dtype=torch.float32)

for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(train_inputs)
    loss = criterion(outputs, train_outputs)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 50 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Save the trained model
#torch.save(model.state_dict(), 'network.txt')

# Step IV - Testing results
test_inputs = torch.tensor(test_data.iloc[:, 2:].values, dtype=torch.float32)
predicted_outputs = model(test_inputs).detach().numpy()

# Save the predictions
pd.DataFrame(predicted_outputs, columns=['Output1', 'Output2']).to_excel('output.xlsx', index=False)

# Display network structure if required
if False:  # Change to False if you don't want to display
    display_network_structure()


Epoch [50/1000], Loss: 0.4817
Epoch [100/1000], Loss: 0.3946
Epoch [150/1000], Loss: 0.3199
Epoch [200/1000], Loss: 0.2406
Epoch [250/1000], Loss: 0.1928
Epoch [300/1000], Loss: 0.1874
Epoch [350/1000], Loss: 0.1868
Epoch [400/1000], Loss: 0.1865
Epoch [450/1000], Loss: 0.1864
Epoch [500/1000], Loss: 0.1862
Epoch [550/1000], Loss: 0.1861
Epoch [600/1000], Loss: 0.1860
Epoch [650/1000], Loss: 0.1859
Epoch [700/1000], Loss: 0.1858
Epoch [750/1000], Loss: 0.1857
Epoch [800/1000], Loss: 0.1857
Epoch [850/1000], Loss: 0.1856
Epoch [900/1000], Loss: 0.1856
Epoch [950/1000], Loss: 0.1855
Epoch [1000/1000], Loss: 0.1854
